## Zajęcia 4: Tokenizacja (text segmentation)

Wszystkie zadania ćwiczeniowe należy rozwiązywać w języku Python w kopii Jupyter Notebook'a dla danych zajęć w wyznaczonych miejscach (komórki z komentarzem `# Solution`).

Nie należy usuwać komórek z treścią zadań.

Należy wyświetlać outputy przy pomocy `print`


### Dla chętnych - super materiał!

tutorial utworzenia tokenizatora wykorzystanego w modelu GPT-2 przez byłego pracownika OpenAI: https://www.youtube.com/watch?v=zduSFxRajkE

Powyższy materiał wyczerpująco opisuje różne zagadnienia tokenizacji - jeżeli ktoś jest bardziej zainteresowany NLP to powinien zapoznać się z tym materiałem.

### Zadanie 1
Wejdź na stronę https://tiktokenizer.vercel.app/ i sprawdź jak wielkie modele języka dzielą tekst na tokeny. Przetestuj teksty z języku polskim oraz angielskim. Napisz różnice między tokenizacją tekstu w języku polskim, a tokenizacją tekstu w języku angielskim.

In [ ]:
# Solution 1 - write the answer as a comment

# Przykladowe wpisane teksty wraz z iloscia tokenow podanych w nawiasach (ilość tokenów jest dla modelu gpt-4o)

# I want you to help me with my homework. (10)
# Chcę zebyś pomógł mi z moim zadaniem domowym. (18)

# Tell me a story about Spongebob Squarepants. (12)
# Opowiedz mi historię o SpongeBobie Kanciastoportym. (16)

# I'm a student and I'm looking for a job. I need money. I need to tank up my car. (22)
# Jestem studentem i szukam pracy. Potrzebuję pieniędzy. Muszę zatankować swój samochód. (28)

# Pillow is really soft. (6)
# Poduszka jest naprawdę miękka. (8)

# Wnioski:
# Pierwszym co rzuca się w oczy jest fakt, 
# że teksty w języku polskim mają o wiele więcej tokenów niż te w języku angielskim.
# Powodem takiego stanu rzeczy jest to, że w języku polskim mamy więcej odmian słów (rzeczownikow,
# czasówników) niż w języku angielskim.
# Np. pies=>psu=>psie=>psem itd., w języku angielskim mamy natomiast tylko dog=>dog's.
# Podsumowując, język polski jest po prostu bardziej złożony niż angielski dla tokenizacji.


### Zadanie 2
Zaimplementuj algorytm MaxMatch na podstawie listy słów z https://www.kaggle.com/datasets/rtatman/english-word-frequency .
Funkcja musi przyjmować słownik oraz tekst z usuniętymi spacjami i zwracać listę słów. Przetestuj program na wybranych 2 tekstach w języku angielskim, które zawierają minimum 100 znaków (spacje z tego tekstu można usunąć automatycznie).



In [19]:
# Solution 2

def maxmatch_text_split(text, vocabulary):
    if len(text) == 0:
        return []
    
    for i in range(len(text),-1,-1):
        word=text[:i]
        remainder=text[i:]
        if word in vocabulary:
            return [word]+maxmatch_text_split(remainder,vocabulary)
    
    # if no word was found return the first letter of sentence as a word
    word=text[0]
    remainder=text[1:]
    return [word]+maxmatch_text_split(remainder,vocabulary)

In [20]:
# Loading the vocabulary
import pandas as pd

readed_csv = pd.read_csv('unigram_freq.csv', header=0)
vocabulary=readed_csv['word'].tolist()

In [21]:
import re

# First text (172 characters) source: lingua.com
text1="""I live in a house near the mountains. I have two brothers and one sister, and I was born last.
My father teaches mathematics, and my mother is a nurse at a big hospital. My brothers are very smart and work hard in school. """

# text1 preparation (make everything lowercase and remove everything that is not a letter or a number (bc everything in provided word list is lowercase) )
text1 = re.sub(r'[^a-zA-Z0-9]', '', text1.lower())

print(maxmatch_text_split(text1,vocabulary))

['ili', 'vein', 'aho', 'usen', 'earth', 'emo', 'unt', 'ainsi', 'have', 'two', 'brothers', 'andon', 'esiste', 'randi', 'was', 'born', 'last', 'myf', 'ather', 'teaches', 'mathematics', 'andm', 'ymo', 'theri', 'sanur', 'seat', 'abi', 'gho', 'spital', 'myb', 'rother', 'sare', 'very', 'smart', 'andw', 'ork', 'hardin', 'school']


In [22]:
# Second text (295 characters) source: lingua.com
text2="""Christina visited Miami during her winter vacation. She is from Boston, where it is cold during the winter months. Miami, however, has a very warm climate.
There are many sunny days in Miami, and people can go to the beach all year long. Christina spent a good portion of her trip on the beach to relax and sunbathe.
However, she also explored Miami and its surroundings."""

# text2 preparation same as for text1
text2 = re.sub(r'[^a-zA-Z0-9]', '', text2.lower())

print(maxmatch_text_split(text2,vocabulary))

['christina', 'visited', 'miami', 'during', 'her', 'winter', 'vacations', 'heis', 'from', 'boston', 'where', 'itis', 'cold', 'duringthe', 'winterm', 'onth', 'smi', 'ami', 'however', 'hasa', 'very', 'warm', 'climate', 'thereare', 'many', 'sunny', 'days', 'inm', 'iam', 'iand', 'people', 'cango', 'tothe', 'beach', 'ally', 'earl', 'ongc', 'hris', 'tinas', 'penta', 'good', 'portion', 'ofh', 'ert', 'ripon', 'thebe', 'acht', 'orel', 'axa', 'ndsu', 'nba', 'theh', 'owever', 'shea', 'lso', 'explored', 'miami', 'andi', 'tss', 'urr', 'ound', 'ings']


### Zadanie 3

Zaimplementuj algorytm BPE na podanym tekście (jako najmniejszą jednostkę należy traktować pojedynczy znak - również spację). Funkcja musi przyjmować tekst oraz maksymalną liczbę elementów w słowniku, a na koniec zwracać tokenizer. Przetestuj program na tekście zawierającym minimum 10000 znaków dla max_vocab_length=500.

Po utworzeniu tokenizera należy dokonać tokenizacji innego tekstu o długości minimum 100 znaków.

UWAGA: tokenizator musi najpierw dokonywać tokenizacji we właściwej kolejności (zaczynając od zmergowanych elementów).

In [23]:
# Solution 3

import re

def prepare_text(text):
    text=re.sub(r'[^a-zA-Z0-9 ]', '', text.lower()) # make everything lowercase and remove everything that is not a letter or a number
    text=text.replace(" ","<w>") # replace all spaces in text with <w> special character
    text=list(text)
    text.insert(0,"<w>")
    text.append("<w>") # add <w> token at the beginning and at the end of the text (we have to make the words have their boundaries)
    return text

def get_initial_vocabulary(text):
    return list(set(text)) # return a vocabulary built from unique characters from text

def get_most_common_bigram_from_text(text):
    bigrams=dict()
    for i in range(0,len(text)-1):
        if text[i]+text[i+1] in bigrams:
            bigrams[text[i]+text[i+1]]+=1
        else:
            bigrams[text[i]+text[i+1]]=1
    sorted_bigrams=sorted(bigrams.items(),key=lambda x:x[1],reverse=True) # sort bigrams by their frequency, but it will preserve the order of bigrams with the same frequency (we have to pick leftmost bigram in case of the same frequency)
    return sorted_bigrams[0][0] # bigrams is dictionary, and we only need to return the bigram (key) itself

def merge_chars(text,bigram_to_merge):
    new_text=[]
    i=0
    while i<len(text)-1:
        if text[i]+text[i+1]==bigram_to_merge:
            new_text.append(bigram_to_merge)
            i+=2
        else:
            new_text.append(text[i])
            i+=1
    return new_text


def create_bpe_tokenizer(text, max_vocab_length):
    
    chars=prepare_text(text)
    vocabulary=get_initial_vocabulary(chars)

    while len(vocabulary)<max_vocab_length:
        if(len(chars)==1): # break from the loop if every character is merged
            break
        
        most_common_bigram=get_most_common_bigram_from_text(chars)
        vocabulary.append(most_common_bigram)
        chars=merge_chars(chars,most_common_bigram)
        
    return vocabulary
    
    
    
def tokenize_text(text, tokenizer):
    chars=prepare_text(text)
    
    sorted_vocabulary=sorted(tokenizer,key=lambda x:len(x),reverse=True) # sort vocabulary by length of token in descending order, to match the longest tokens first
    
    tokenized_text=[]
    i=0
    while i<len(chars):
        found=False
        for token in sorted_vocabulary:
            if chars[i:i+len(token)]==list(token): # we need to convert token (string type) to list in order to compare it with a slice of chars
                tokenized_text.append(token)
                i+=len(token)
                found=True
                break
        if not found:
            tokenized_text.append(chars[i])
            i+=1
    
    # replace <w> tokens with spaces 
    for i in range(len(tokenized_text)):
        if "<w>" in tokenized_text[i]:
            tokenized_text[i]=tokenized_text[i].replace("<w>"," ")
            
    return tokenized_text

In [27]:
# text is sum of two prompts to gpt-4o (gpt-4o refused to generate text longer than 9100 characters in one prompt)
# 1st prompt: "Create an article about F1 that takes minimum 10000 characters. Don't make paragraphs, just write a continuous text."
# 2nd prompt: "Continue, that is not 10000 characters yet."
# length: 11469

initial_text="""Formula 1, often abbreviated as F1, represents the pinnacle of single-seater, open-wheel racing and is widely regarded as the premier class of motorsport.
Governed by the Fédération Internationale de l'Automobile (FIA), F1 has a rich history that stretches back over seventy years, with the first official World Championship race held at Silverstone, England, in 1950.
F1 attracts the best drivers, engineers, and teams, all of whom are dedicated to pushing the boundaries of automotive technology and human performance.
The sport’s hallmark is its blend of speed, precision, and engineering excellence, with cars reaching speeds of over 350 kilometers per hour (217 mph) on some tracks.
The FIA sets strict regulations, from technical specifications to race-day protocols, to ensure safety, competition, and fairness, making F1 a highly competitive arena where even the smallest details can determine the outcome of a championship season.
The structure of F1 consists of ten teams, each fielding two cars, for a total of twenty drivers competing on some of the most famous tracks around the world, from the high-speed straights of Monza in Italy to the tight corners of Monaco's street circuit.
Teams such as Ferrari, McLaren, Mercedes, and Red Bull Racing are iconic names in the sport, with decades of history, legions of fans, and an impressive number of world titles.
Every season consists of multiple Grand Prix races—historically between 16 and 23 races, though this number fluctuates based on logistics and planning.
Each race weekend is structured around practice sessions, a qualifying session, and the race itself.
The points awarded for race positions contribute to the drivers' and constructors' championships, making each position critical as drivers and teams aim for the best possible finish.
Over the years, F1 has evolved dramatically, both technologically and culturally.
In the early days of F1, cars were basic compared to today's standards, with minimal attention to aerodynamics, safety measures, and driver comfort. As technology advanced, so did the complexity and performance of F1 cars.
The 1960s saw the introduction of rear-mounted engines, which revolutionized car design and significantly increased speeds.
By the 1970s and 1980s, aerodynamics became a core focus, with teams using wind tunnels to refine car shapes and improve downforce.
The concept of ground effect was introduced, allowing cars to generate additional downforce by shaping the underside of the vehicle to create a low-pressure area.
This era also saw advancements in safety measures, with the addition of seatbelts, roll bars, and eventually fire-resistant suits for drivers.
The 1990s and early 2000s were marked by rapid technological advancements, including the introduction of electronic aids like traction control, semi-automatic gearboxes, and advanced telemetry systems that allowed teams to monitor car performance in real-time.
However, these developments led to concerns that the cars were becoming too reliant on technology, diminishing the emphasis on driver skill.
Consequently, the FIA introduced regulations to limit electronic aids, ensuring that F1 remained a driver-focused sport.
In recent years, F1 has embraced hybrid technology to address environmental concerns and push the boundaries of automotive efficiency.
The current power units are hybrid engines, combining a 1.6-liter turbocharged V6 engine with an energy recovery system (ERS) that captures and reuses energy generated during braking and acceleration.
This system not only enhances performance but also reduces fuel consumption, making F1 one of the most fuel-efficient racing series despite its high speeds.
Additionally, the FIA has set ambitious goals to make F1 carbon-neutral by 2030, driving further innovation in sustainable technology.
Another significant aspect of F1 is its global reach and fan engagement. F1 has a massive following worldwide, with millions of fans tuning in to watch races on television and streaming platforms.
The sport has also embraced digital media, with active social media channels, a dedicated streaming service (F1 TV), and a popular Netflix series, "Drive to Survive," which has introduced F1 to new audiences and increased its popularity, especially in the United States.
Fans are drawn to F1 not only for the on-track action but also for the personalities and rivalries that define the sport.
Legendary drivers such as Juan Manuel Fangio, Ayrton Senna, Alain Prost, Michael Schumacher, and more recently Lewis Hamilton and Max Verstappen, have captivated audiences with their skill, determination, and charisma.
Rivalries between drivers and teams have become legendary, often creating narratives that transcend the sport.
One of the most famous rivalries was between Ayrton Senna and Alain Prost in the late 1980s and early 1990s, which saw intense competition and on-track clashes that are still remembered by fans today.
In recent years, the rivalry between Lewis Hamilton and Max Verstappen has brought new excitement to the sport, with the two drivers battling for the world championship in closely contested races.
The relationship between F1 and technology is symbiotic, with advancements in F1 often influencing consumer automotive technology.
Many features now standard in road cars, such as anti-lock braking systems (ABS), traction control, and hybrid engines, were developed and refined in F1 before being adapted for consumer use.
F1 teams invest heavily in research and development, often spending hundreds of millions of dollars each year to gain a competitive edge.
This investment drives innovation, resulting in cars that are not only faster but also safer and more efficient.
F1 has also become a platform for experimenting with new materials, such as carbon fiber, which is now widely used in both racing and consumer vehicles due to its strength and light weight.
Safety is a paramount concern in F1, and the sport has made significant strides in protecting drivers and reducing the risk of fatal accidents.
The introduction of the halo device, a protective structure around the cockpit, has been one of the most significant recent safety innovations.
Initially controversial, the halo has since proven its worth, preventing serious injuries in several high-profile accidents.
Other safety measures include improved helmets, fire-resistant clothing, and advanced crash structures designed to absorb impact forces.
The FIA also enforces strict protocols for circuit design, ensuring that tracks have adequate run-off areas, barriers, and medical facilities.
F1 teams are a unique combination of engineering expertise, financial investment, and strategic decision-making.
Each team employs hundreds of people, including engineers, mechanics, strategists, and support staff, all working towards the common goal of winning races and championships.
The role of the team principal is crucial, as they are responsible for overseeing the team’s operations, making strategic decisions, and managing relationships with drivers, sponsors, and the FIA.
Key figures in F1 team management, such as Toto Wolff (Mercedes), Christian Horner (Red Bull Racing), and Zak Brown (McLaren), have become well-known personalities in their own right, often involved in on-track rivalries and off-track controversies.
The role of the driver is equally important, as they must possess not only exceptional skill and reflexes but also the mental resilience to handle the pressures of competition, media scrutiny, and the demands of a grueling race calendar.
F1 drivers undergo rigorous physical training to prepare for the extreme conditions they face in the cockpit, where temperatures can exceed 50 degrees Celsius (122 degrees Fahrenheit) and G-forces during cornering can reach up to 5G.
In addition to physical fitness, drivers must have a deep understanding of their cars, working closely with engineers to optimize performance and provide feedback on handling and setup.
The future of F1 is likely to be shaped by several key trends, including sustainability, technological innovation, and efforts to increase diversity and inclusion within the sport.
The push towards carbon neutrality is driving research into synthetic fuels and other eco-friendly technologies that could eventually replace traditional gasoline engines.
F1 is also exploring ways to make the sport more accessible and affordable, as rising costs have made it difficult for smaller teams to compete.
The introduction of a cost cap in recent years is one step towards addressing this issue, as it limits the amount teams can spend on certain aspects of car development.
F1 is also working to improve diversity, both on and off the track.
Initiatives such as the FIA’s “We Race as One” campaign aim to promote inclusivity and encourage people from underrepresented backgrounds to pursue careers in motorsport.
Lewis Hamilton, the sport's only Black driver and a seven-time world champion, has been a vocal advocate for diversity, using his platform to raise awareness of social and environmental issues.
Formula 1 is increasingly focused on expanding its fan base by reaching new audiences in emerging markets, notably in Asia and the United States, with efforts to grow a global presence that matches its worldwide fan interest.
With races traditionally hosted in Europe and a few other regions, F1 is now venturing into new territories, recently adding races in places like Miami, Saudi Arabia, and returning to historic venues like the Dutch Grand Prix in Zandvoort.
This international expansion is strategically aimed at establishing F1 as a truly global sport, leveraging the power of regional fanbases and new-age digital platforms to engage people worldwide.
The American market has become particularly vital for F1, especially with the success of Netflix’s “Drive to Survive” series, which has captivated new fans and sparked increased demand for more races in the United States.
In 2023, Las Vegas became the host of a glamorous night race, marking one of the latest efforts to build on F1’s popularity in North America and showcase its appeal in the entertainment capital of the world.
At the heart of F1’s allure are its cars, which are marvels of engineering built to operate at the limits of performance.
Each car is a masterpiece, with every component carefully designed to be as light and strong as possible. 
The car’s aerodynamics, tires, chassis, and power unit must work in perfect harmony to maximize speed and handling.
Aerodynamics play a crucial role, with each car designed to generate downforce, which helps it grip the track at high speeds.
The balance of aerodynamics is so critical that even minor modifications can result in measurable changes in lap time.
F1 teams invest millions in developing these components, utilizing sophisticated wind tunnels and computer simulations to refine designs before they reach the track.
Beyond aerodynamics, the tires used in F1 are another essential factor that can make or break a race strategy. Pirelli is the exclusive tire supplier for F1, providing teams with a range of compounds that vary in grip and durability.
Drivers and teams must carefully manage their tires during a race, balancing speed with conservation, as aggressive driving can lead to quicker tire degradation.
Pit stop strategy is crucial here, with teams employing specialized crews trained to change all four tires in a matter of seconds.
"""

tokenizer=create_bpe_tokenizer(initial_text, 500)

# text generated by gpt-4o prompt (follow up to previous prompt): "Now in the same style generate text about Max Verstappen 2021 season that takes about 250 characters"
text_to_tokenize="""Max Verstappen's 2021 season was a breakthrough, showcasing relentless skill and resilience.
Driving for Red Bull Racing, he battled fiercely with Lewis Hamilton, achieving ten wins and consistent podiums.
His season culminated in a dramatic final-lap victory in Abu Dhabi, securing his first World Championship."""


print(tokenize_text(text_to_tokenize, tokenizer))

[' ', 'ma', 'x ', 'verst', 'ap', 'pe', 'n', 's ', '2', '0', '2', '1 ', 'se', 'as', 'on ', 'w', 'as ', 'a ', 'b', 're', 'a', 'k', 'th', 'ro', 'u', 'gh', ' ', 'sh', 'o', 'w', 'c', 'as', 'ing ', 're', 'le', 'n', 't', 'le', 's', 's ', 's', 'kil', 'l ', 'and ', 're', 'si', 'li', 'enc', 'ed', 'ri', 'vi', 'n', 'g', ' ', 'for ', 'red ', 'bu', 'l', 'l ', 'racing ', 'h', 'e ', 'b', 'at', 't', 'le', 'd ', 'fi', 'er', 'c', 'el', 'y ', 'with ', 'le', 'wi', 's h', 'am', 'il', 'to', 'n', ' ', 'ach', 'i', 'e', 'vi', 'n', 'g', ' ', 'ten ', 'wi', 'n', 's and ', 'con', 'sist', 'ent ', 'po', 'di', 'um', 'shi', 's ', 'se', 'as', 'on ', 'c', 'ul', 'min', 'ated ', 'in ', 'a ', 'd', 'ra', 'ma', 'tic', ' ', 'fin', 'all', 'ap', ' ', 'vi', 'c', 'to', 'r', 'y ', 'in ', 'ab', 'u', ' ', 'd', 'ha', 'b', 'i', ' ', 'se', 'c', 'ur', 'ing ', 'hi', 's ', 'fi', 'r', 'st ', 'world ', 'championshi', 'p', ' ']
